In [1]:
import numpy as np

## Linear Case

$$
\tau(n) = \frac{1}{n(n-1)}\sum_{u+v=n}(au+bv)\tau(u)\tau(v)
$$
Due to symmetry, we can rewrite the sum as
$$
\tau(n) = \frac{1}{n(n-1)}\sum_{u+v=n}au\tau(u)\tau(v).
$$

In [2]:
def deg_1_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_array = np.multiply.outer(a_range,u_range) # a x u
    bv_array = np.multiply.outer(b_range,v_range) # b x v
    
    temp_array = np.add.outer(au_array, bv_array) # a x u x b x v
    
    final_array = np.swapaxes(temp_array, 1,2)
    
    return final_array


def deg_1_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_1_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/(n*(n-1)))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [3]:
deg_1_sqnc(-5,-5,4,4,21)

array([[[ 0.00000000e+00,  1.00000000e+00, -5.00000000e-01,
          2.50000000e-01, -1.25000000e-01,  6.25000000e-02,
         -3.12500000e-02,  1.56250000e-02, -7.81250000e-03,
          3.90625000e-03, -1.95312500e-03,  9.76562500e-04,
         -4.88281250e-04,  2.44140625e-04, -1.22070312e-04,
          6.10351562e-05, -3.05175781e-05,  1.52587891e-05,
         -7.62939453e-06,  3.81469727e-06, -1.90734863e-06]]])

## Quadratic Case

In [4]:
def deg_2_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range) # a x u
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length)) # k x b x a x k
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range) # b_len x seq_length
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b_len x seq_len x seq_len
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length))
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    #corrected_buv_tensor = np.swapaxes(buv_tensor, 2,3)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array


def deg_2_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_2_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/((n*n)*(n-1)))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [5]:
deg_2_sqnc(-24,-24,36,36,21)

array([[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
         28., 14., 24., 24., 31., 18., 39., 20., 42.]]])

## Cubic Case

In [6]:
def deg_3_array(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    a_range = np.arange(a_min, a_max+1)
    b_range = np.arange(b_min, b_max+1)
    u_range = np.arange(seq_length)
    v_range = u_range
    
    au_squared_matrix = np.multiply.outer(a_range, u_range*u_range*u_range) # a x u^3
    au_tensor = np.broadcast_to(au_squared_matrix,(seq_length,b_len,a_len,seq_length)) # k x b x a x k
    #k,b,a,k --> a,b,k,k
    
    bu_matrix = np.multiply.outer(b_range,u_range*u_range) # b x u^2
    buv_matrix = np.multiply.outer(bu_matrix,v_range) # b x u^2 x v
    buv_tensor = np.broadcast_to(buv_matrix,(a_len,b_len,seq_length,seq_length)) # a x b x k x k
    
    corrected_au_tensor = np.swapaxes(au_tensor, 0,2)
    #corrected_buv_tensor = np.swapaxes(buv_tensor, 2,3)
    
    tensor = corrected_au_tensor + buv_tensor
    
    P_array = np.swapaxes(tensor, 2,3)
    
    return P_array


def deg_3_sqnc(a_min,a_max,b_min,b_max,seq_length):
    a_len = a_max+1 - a_min
    b_len = b_max+1 - b_min
    
    coeff_array = deg_3_array(a_min,a_max,b_min,b_max,seq_length)
    seq = np.zeros(seq_length)
    seq[1] = 1
    seq_array = np.broadcast_to(seq, (a_len,b_len,seq_length)).copy()
    
    flipped_array = coeff_array[:,:,::-1,:]
    for n in range(2,seq_length):
        diag = np.diagonal(flipped_array, -seq_length+n+1, axis1 = 2, axis2 = 3)[:,:,1:n]
        vec_A = diag*seq_array[:,:,1:n]
        vec_B = seq_array[:,:,n-1:0:-1]
        seq_array[:,:,n] = (1/((n**3)*(n-1)))*np.sum(vec_A*vec_B,axis=2)
    
    return seq_array

In [7]:
deg_3_sqnc(6,6,18,18,21)

array([[[0.00000000e+00, 1.00000000e+00, 3.00000000e+00, 9.00000000e+00,
         2.70000000e+01, 8.10000000e+01, 2.43000000e+02, 7.29000000e+02,
         2.18700000e+03, 6.56100000e+03, 1.96830000e+04, 5.90490000e+04,
         1.77147000e+05, 5.31441000e+05, 1.59432300e+06, 4.78296900e+06,
         1.43489070e+07, 4.30467210e+07, 1.29140163e+08, 3.87420489e+08,
         1.16226147e+09]]])

## Catalan Numbers

In [8]:
def catalan(length):
    catlist = np.zeros(length)
    catlist[1] = 1
    for n in range(2,length):
        vec_A = catlist[1:n]
        vec_B = catlist[n-1:0:-1]
        catlist[n] = np.dot(vec_A,vec_B)

    return catlist

In [9]:
catalan(21)

array([0.00000000e+00, 1.00000000e+00, 1.00000000e+00, 2.00000000e+00,
       5.00000000e+00, 1.40000000e+01, 4.20000000e+01, 1.32000000e+02,
       4.29000000e+02, 1.43000000e+03, 4.86200000e+03, 1.67960000e+04,
       5.87860000e+04, 2.08012000e+05, 7.42900000e+05, 2.67444000e+06,
       9.69484500e+06, 3.53576700e+07, 1.29644790e+08, 4.77638700e+08,
       1.76726319e+09])

## Checking for Geometric Sequences

In [10]:
def array_div(A): # input is 3d array
    orig_sqnc_len = A.shape[2]
    
    shift_1 = A[:,:,2::] # starts each sequence at their first index
    shift_2 = A[:,:,1:-1] # then at their second index
    
    div_array = np.divide(shift_1,shift_2, out=np.full_like(shift_1,np.nan), where=shift_2!=0)
    # puts a zero wherever it had to divide by zero
    
    return div_array

In [11]:
def geom_sqnc_check(A,a_min=0,b_min=0): # need those extra inputs to get the a and b values correct in the tuples
    np.seterr(all="ignore")
   
    quot_array = array_div(A)
    bool_array = (np.abs(quot_array[:,:,:-1] - quot_array[:,:,1:]) < 1e-5).sum(axis=2) == (quot_array.shape[2]-1)

    arrays = np.where(bool_array)
    x_coords = arrays[0]
    y_coords = arrays[1]
    length = len(x_coords)
    coord_list = [(x_coords[n]+a_min,y_coords[n]+b_min) for n in range(length)]
    coord_list
    
    return coord_list

In [12]:
geom_sqnc_check(deg_1_sqnc(-10,10,-10,10,21),-10,-10)[:25]

[(-10, -10),
 (-10, -9),
 (-10, -8),
 (-10, -7),
 (-10, -6),
 (-10, -5),
 (-10, -4),
 (-10, -3),
 (-10, -2),
 (-10, -1),
 (-10, 0),
 (-10, 1),
 (-10, 2),
 (-10, 3),
 (-10, 4),
 (-10, 5),
 (-10, 6),
 (-10, 7),
 (-10, 8),
 (-10, 9),
 (-9, -10),
 (-9, -9),
 (-9, -8),
 (-9, -7),
 (-9, -6)]

In [13]:
array_div(deg_1_sqnc(1000,1000,-327,-327,21)) # a = 1000, b = -327

array([[[336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5,
         336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5, 336.5,
         336.5]]])

It looks like the linear case *only* produces geometric sequences!

In [14]:
geom_sqnc_check(deg_2_sqnc(-10,10,-10,10,21),-10,-10)

[(-10, -10),
 (-9, -9),
 (-8, -8),
 (-7, -7),
 (-6, -6),
 (-5, -5),
 (-4, -4),
 (-3, -3),
 (-2, -2),
 (-1, -1),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9),
 (10, 10)]

In [15]:
array_div(deg_2_sqnc(-10,10,-10,10,21))[14,14]

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2.])

In [16]:
array_div(deg_2_sqnc(-10,10,-10,10,21))[16,16]

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3.])

Looks like the quadratic function produces a geometric sequence for each ordered pair $(a,b) = (a,a)$ and sequences of zeros for $(a,b) = (a,-a)$, $a \in \mathbb{Z}$.

In [17]:
geom_sqnc_check(deg_3_sqnc(-30,30,-50,50,21),-30,-50)

[(-16, -48),
 (-15, -45),
 (-14, -42),
 (-13, -39),
 (-12, -36),
 (-11, -33),
 (-10, -30),
 (-9, -27),
 (-8, -24),
 (-7, -21),
 (-6, -18),
 (-5, -15),
 (-4, -12),
 (-3, -9),
 (-2, -6),
 (-1, -3),
 (1, 3),
 (2, 6),
 (3, 9),
 (4, 12),
 (5, 15),
 (6, 18),
 (7, 21),
 (8, 24),
 (9, 27),
 (10, 30),
 (11, 33),
 (12, 36),
 (13, 39),
 (14, 42),
 (15, 45),
 (16, 48)]

In [18]:
deg_3_sqnc(-24,-24,48,48,21) # finding our original sigma_n in the cubics

array([[[ 0.,  1.,  3.,  4.,  7.,  6., 12.,  8., 15., 13., 18., 12.,
         28., 14., 24., 24., 31., 18., 39., 20., 42.]]])

## Some Ideas

After finding a function that finds the geometric sequences, try dividing everything by sequences of catalan numbers or sequences of sums of factors, then put it through the function. This way we can pinpoint the combo-sequences.

### Finding Combo Sequences with $\sigma_n$

In [19]:
divsigma = deg_2_sqnc(-40,40,-40,40,21)
sigma = deg_2_sqnc(-24,-24,36,36,21)
divsigma[:,:,1::] = divsigma[:,:,1::]/sigma[:,:,1::]

geom_sqnc_check(divsigma,-40,-40)

[(-26, 39),
 (-24, 36),
 (-22, 33),
 (-20, 30),
 (-18, 27),
 (-16, 24),
 (-14, 21),
 (-12, 18),
 (-10, 15),
 (-8, 12),
 (-6, 9),
 (-4, 6),
 (-2, 3),
 (2, -3),
 (4, -6),
 (6, -9),
 (8, -12),
 (10, -15),
 (12, -18),
 (14, -21),
 (16, -24),
 (18, -27),
 (20, -30),
 (22, -33),
 (24, -36),
 (26, -39)]

First look at possible combo-sequences:

In [20]:
array_div(divsigma)[14,79] # a = -26, b = 39

array([1.08333333, 1.08333333, 1.08333333, 1.08333333, 1.08333333,
       1.08333333, 1.08333333, 1.08333333, 1.08333333, 1.08333333,
       1.08333333, 1.08333333, 1.08333333, 1.08333333, 1.08333333,
       1.08333333, 1.08333333, 1.08333333, 1.08333333])

Looks like $\big\{\left(\frac{13}{12}\right)^n\big\}$ for $a = -26$ and $b = 39$. Now to look at the original sequence in that place:

In [21]:
deg_2_sqnc(-40,40,-40,40,21)[14,79]

array([  0.        ,   1.        ,   3.25      ,   4.69444444,
         8.89988426,   8.26417824,  17.90571952,  12.93190854,
        26.26793923,  24.66267628,  36.99401441,  26.7178993 ,
        67.53691212,  36.58249406,  67.93891755,  73.60049401,
       102.98958016,  64.78376816, 152.06190027,  84.47883348,
       192.18934618])

In [22]:
array_div(deg_2_sqnc(-40,40,-40,40,21))[52,22]

array([-1.5       , -0.66666667, -0.875     , -0.42857143, -1.        ,
       -0.33333333, -0.9375    , -0.43333333, -0.69230769, -0.33333333,
       -1.16666667, -0.25      , -0.85714286, -0.5       , -0.64583333,
       -0.29032258, -1.08333333, -0.25641026, -1.05      ])

In [23]:
array_div(divsigma)[52,22]

array([-0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5,
       -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5])

Looks pretty evident that $a = 12$, $b = -18$ $\Rightarrow$ $\{-0.5^n\}\times\sigma(n)$. Now to find some more:

In [24]:
array_div(divsigma)[22,67]

array([0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75,
       0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75])

$a = -18$, $b = 27$ $\Rightarrow$ $\{0.75^n\}\times\sigma(n)$.

In [25]:
array_div(divsigma)[34, 49]

array([0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25])

$a = -6, b = 9$ $\Rightarrow$ $\{0.25^n\}\times\sigma(n)$

In [26]:
array_div(divsigma)[16, 76] # this is just where sigma_n would be

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1.])

In [27]:
array_div(divsigma)[38,43] # 0.08333333... = 1/12

array([0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.08333333,
       0.08333333, 0.08333333, 0.08333333, 0.08333333])

$a = -2, \ b = 3 \ \Rightarrow \ \big\{\left(\frac{1}{12}\right)^n\big\}\times\sigma(n)$. Didn't want to show up from the function because of floating point inconsistencies.

In [28]:
array_div(divsigma)[28,58]

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

$a = -12$, $b = 18$ $\Rightarrow$ $\{0.5^n\}\times\sigma(n)$

In [29]:
array_div(divsigma)[58,13]

array([-0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75,
       -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75,
       -0.75])

$a = 18$, $b = -27$ $\Rightarrow$ $\{-0.75^n\}\times\sigma(n)$.

In [30]:
array_div(divsigma)[36,46] # 0.166666... = 1/6

array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667, 0.16666667, 0.16666667, 0.16666667])

$a = -4, \ b = 6 \ \Rightarrow \big\{\left(\frac{1}{6}\right)^n\big\}\times\sigma(n)$

What I'm seeing is that, for $\tau(n) = \frac{1}{n^2(n-1)}\sum_{u+v=n}(au^2+buv)\tau(u)\tau(v)$, $(a,b)=(a,a)$ produces a geometric sequence $\{\rho^n\}$ for $\rho \in \mathbb{Q}$, $(a,b) = (-24,36)$ produces $\sigma(n)$, and $(a,b) = (-2k,3k)$ for $k \in \mathbb{Z}$ produces some sequence $\{\rho^n\}\times\sigma(n)$. The next step would be to connect $a$ and $\rho$ as well as $k$ and $\rho$.

### Next Stuff

Play with other denominators, like $\frac{1}{n^3}$ or $\frac{1}{n(n-1)(n-2)}$. Also extend search to higher degree polynomials - quartics, quintics, etc.